In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if not os.path.exists('deepSORT_MOT17'):
  !git clone https://github.com/Ulyssesllc/deepSORT_MOT17.git
else:
  !git -C deepSORT_MOT17 pull --ff-only

Cloning into 'deepSORT_MOT17'...
remote: Enumerating objects: 11486, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 11486 (delta 9), reused 16 (delta 6), pack-reused 11465 (from 1)
Receiving objects: 100% (11486/11486), 1.82 GiB | 43.22 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
# Sync MOT17 directly from Drive into the repo to keep the repo slim
repo_root = '/content/deepSORT_MOT17'
drive_zip = '/content/drive/MyDrive/MOT17.zip'
drive_dir = '/content/drive/MyDrive/MOT17'

import os
os.makedirs(repo_root, exist_ok=True)

# Prefer syncing the folder if it exists (faster, incremental). Otherwise unzip the ZIP directly into the repo.
if os.path.exists(drive_dir):
  print('Syncing MOT17 folder from Drive ->', f'{repo_root}/MOT17')
  !rsync -a --delete "/content/drive/MyDrive/MOT17/" "/content/deepSORT_MOT17/MOT17/"
elif os.path.exists(drive_zip):
  print('Unzipping MOT17.zip from Drive into repo ->', repo_root)
  !unzip -q "/content/drive/MyDrive/MOT17.zip" -d "/content/deepSORT_MOT17"
else:
  raise SystemExit('MOT17 source not found on Drive. Provide MOT17/ folder or MOT17.zip in MyDrive.')

# Verify a few entries
print('Top-level sequences under repo MOT17/:')
!ls -1 "/content/deepSORT_MOT17/MOT17" | head -n 10

In [ ]:
import os

os.chdir('deepSORT_MOT17')
!pip install -r requirements.txt
# Ensure TF packages available on Colab for feature extraction
!pip install tensorflow tf-keras tf-slim
# Ensure ffmpeg is present for video conversion
!apt-get update -y && apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 5.0 MB/s eta 0:00:00


In [5]:
# Ensure the MARS ReID model exists at expected path
import pathlib
model_dst = pathlib.Path('resources/networks/mars-small128.pb')
model_dst.parent.mkdir(parents=True, exist_ok=True)
# If you have it on Drive, copy it; else download from your own link
# Example (adjust the source path accordingly):
# !cp /content/drive/MyDrive/models/mars-small128.pb resources/networks/mars-small128.pb
print('Model exists:', model_dst.exists(), '->', model_dst)

Model exists: False -> resources/networks/mars-small128.pb


In [6]:
# Optional: validate after features are generated. Skip now to avoid failing before creation.
# !python validate_paths.py \
#   --sequence_dir MOT17/train/MOT17-04-FRCNN \
#   --detection_file detections/MOT17-04-FRCNN.npy
print('Skipping validation pre-generation. Will validate after features are created.')

Skipping validation pre-generation. Will validate after features are created.


In [7]:
!python tools/generate_detections.py \
  --model /content/deepSORT_MOT17/resources/networks/mars-small128.pb \
  --mot_dir MOT17/train \
  --output_dir detections

2025-08-28 08:44:52.422662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756370692.788804    2444 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756370692.885716    2444 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756370693.613369    2444 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756370693.613410    2444 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756370693.613417    2444 computation_placer.cc:177] computation placer alr

In [8]:
# Also generate features for the test split (no gt there)
!python tools/generate_detections.py \
  --model /content/deepSORT_MOT17/resources/networks/mars-small128.pb \
  --mot_dir MOT17/test \
  --output_dir detections

2025-08-28 08:45:04.836876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756370704.871068    2537 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756370704.881456    2537 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756370704.906064    2537 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756370704.906105    2537 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756370704.906115    2537 computation_placer.cc:177] computation placer alr

In [9]:
# Now that features exist, validate one sequence path and its .npy
!python validate_paths.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy

[OK ] sequence_dir: MOT17/train/MOT17-04-FRCNN
  - img1: OK
  - det/det.txt: OK
[FAIL] Không tìm thấy detection_file: detections/MOT17-04-FRCNN.npy

Gợi ý:
- Đặt dữ liệu theo cấu trúc: MOT17/train/<sequence>/{img1/, det/det.txt, (gt/gt.txt)}
- Tạo đặc trưng bằng: python tools/generate_detections.py --mot_dir MOT17/train --output_dir detections


In [10]:
!python evaluate_motchallenge.py \
  --mot_dir MOT17/train \
  --detection_dir detections \
  --output_dir results \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100

[WARN] Missing detections for MOT17-02-DPM, expected: detections/MOT17-02-DPM.npy. Skipping.
[WARN] Missing detections for MOT17-02-FRCNN, expected: detections/MOT17-02-FRCNN.npy. Skipping.
[WARN] Missing detections for MOT17-02-SDP, expected: detections/MOT17-02-SDP.npy. Skipping.
[WARN] Missing detections for MOT17-04-DPM, expected: detections/MOT17-04-DPM.npy. Skipping.
[WARN] Missing detections for MOT17-04-FRCNN, expected: detections/MOT17-04-FRCNN.npy. Skipping.
[WARN] Missing detections for MOT17-04-SDP, expected: detections/MOT17-04-SDP.npy. Skipping.
[WARN] Missing detections for MOT17-05-DPM, expected: detections/MOT17-05-DPM.npy. Skipping.
[WARN] Missing detections for MOT17-05-FRCNN, expected: detections/MOT17-05-FRCNN.npy. Skipping.
[WARN] Missing detections for MOT17-05-SDP, expected: detections/MOT17-05-SDP.npy. Skipping.
[WARN] Missing detections for MOT17-09-DPM, expected: detections/MOT17-09-DPM.npy. Skipping.
[WARN] Missing detections for MOT17-09-FRCNN, expected: de

In [ ]:
# (Optional) Run tracker on test split as well (no GT there)
!python evaluate_motchallenge.py \
  --mot_dir MOT17/test \
  --detection_dir detections \
  --output_dir results \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100

In [11]:
# Evaluate metrics on train split only (test split has no GT)
!python evaluate_metrics.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --output_csv mot17_train_metrics.csv \
  --min_iou 0.5

[WARN] Bỏ qua MOT17-02-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-02-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-02-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-04-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-04-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-04-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-05-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-05-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-05-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-09-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-09-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-09-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-10-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-10-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-10-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-11-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-11-FRCNN: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-11-SDP: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17-13-DPM: thiếu gt hoặc kết quả.
[WARN] Bỏ qua MOT17

In [12]:
!python deep_sort_app.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --output_file results/MOT17-04-FRCNN.txt \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100 \
  --display False

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 332, in <module>
    run(
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 200, in run
    seq_info = gather_sequence_info(sequence_dir, detection_file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 69, in gather_sequence_info
    raise FileNotFoundError(
FileNotFoundError: Không tìm thấy detection_file: detections/MOT17-04-FRCNN.npy. Hãy chắc chắn đã tạo bằng tools/generate_detections.py.


In [13]:
# Visualize results: on Colab, this will render frames to a window-less viewer; use video export + inline preview below for display
!python show_results.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --result_file results/MOT17-04-FRCNN.txt \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --update_ms 20

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/show_results.py", line 128, in <module>
    run(
  File "/content/deepSORT_MOT17/show_results.py", line 43, in run
    seq_info = deep_sort_app.gather_sequence_info(sequence_dir, detection_file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 69, in gather_sequence_info
    raise FileNotFoundError(
FileNotFoundError: Không tìm thấy detection_file: detections/MOT17-04-FRCNN.npy. Hãy chắc chắn đã tạo bằng tools/generate_detections.py.


In [ ]:
!python generate_videos.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --detection_dir detections \
  --output_dir videos \
  --update_ms 20 \
  --convert_h264

In [ ]:
# Inline preview the generated video on Colab
import os
from IPython.display import HTML
from base64 import b64encode
video_path = 'videos/MOT17-04-FRCNN.mp4'
if not os.path.exists(video_path):
  # Fallback to a likely name; list videos
  import glob
  vids = glob.glob('videos/*.mp4')
  print('Videos found:', vids)
  if vids:
    video_path = vids[0]
with open(video_path,'rb') as f:
  mp4 = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Videos found: []


FileNotFoundError: [Errno 2] No such file or directory: 'videos/MOT17-04-FRCNN.mp4'